In [21]:
import dataAck
import portfolio
import time
import hashlib
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [4]:
##GET PREDICTIONS OF ALL MODELS IN A PORTFOLIO

allPortfolios = portfolio.getPortfolios()
allPortfolios

[{'benchmark': 'VO',
  'description': 'VO 8-13-17',
  'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f'}]

In [5]:
allModels = []
modelsInPortfolio = {}
for portfolioInfo in allPortfolios:
    print(portfolioInfo)
    models = portfolio.getModelsByKey(portfolio.getPortfolioModels(portfolioInfo["key"]))
    modelsInPortfolio[portfolioInfo["key"]] = models
    allModels += models
    for model in models:
        print(model.describe())
    

{'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}
((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)
((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)


In [6]:
tickersRequired = []
tickersTraded = []
for mod in allModels:
    print(mod.describe())
    if mod.inputSeries.targetTicker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.targetTicker)
    if mod.inputSeries.series.ticker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.series.ticker)
    if mod.inputSeries.targetTicker not in tickersTraded:
        tickersTraded.append(mod.inputSeries.targetTicker)
    
pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])

((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)
((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)
TIP
VO
SHY
IWF
XLV


In [9]:
from google.cloud import datastore, storage, logging
import time
import params
import hashlib
import pandas as pd
def downloadAggregatePredictions(model):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.aggregatePrediction)
            
            query.add_filter('modelHash', '=', hashlib.sha224((str(model.describe())).encode('utf-8')).hexdigest())
            retrievedPredictions = list(query.fetch())
            days = []
            predictions = []
            for pred in retrievedPredictions:
                days.append(pred["predictionDay"])
                predictions.append(pred["aggregatePrediction"])
            
            return pd.DataFrame(predictions, index=days, columns=[str(model.describe())]).sort_index()
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [97]:
aggregateReturns = None
aggregatePredictions = None
for model in allModels:
    preds = downloadAggregatePredictions(model).tz_localize(None)
    dailyFactorReturn = dataAck.getDailyFactorReturn(model.inputSeries.targetTicker, joinedData)
    transformedPreds = preds.join(dailyFactorReturn).dropna()
    returnStream = pd.DataFrame(transformedPreds.apply(lambda x:x[0] * x[1], axis=1), columns=[str(model.describe())])
    preds.columns = [str(model.describe())]
    if aggregateReturns is None:
        aggregateReturns = returnStream
        aggregatePredictions = preds
    else:
        aggregateReturns = aggregateReturns.join(returnStream)
        aggregatePredictions = aggregatePredictions.join(preds)
aggregateReturns

,"((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)","((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)","((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)","((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)","((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)"
2006-11-06,0.000000,-0.002369,-0.002369,0.000000,0.004737
2006-11-07,0.003772,-0.003772,-0.001886,0.003772,0.003772
2006-11-08,-0.002349,0.002349,-0.000000,-0.002349,-0.002349
2006-11-09,0.002197,-0.002197,0.000000,0.002197,0.001099
2006-11-10,0.001879,-0.001879,-0.000940,0.000000,0.000000
2006-11-13,0.010628,0.000000,-0.005314,0.000000,-0.005314
2006-11-14,0.000000,0.002011,-0.004021,0.000000,-0.004021
2006-11-15,-0.000000,0.000000,-0.000000,-0.000000,-0.000000
2006-11-16,0.000000,0.000000,-0.000462,-0.000924,-0.000924
2006-11-17,0.000000,-0.000616,0.000000,0.000000,-0.001231


In [98]:
aggregatePredictions

,"((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)","((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)","((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)","((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)","((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)"
2006-11-06,0.0,-0.5,-0.500000,0.0,1.0
2006-11-07,1.0,-1.0,-0.500000,1.0,1.0
2006-11-08,1.0,-1.0,0.000000,1.0,1.0
2006-11-09,1.0,-1.0,0.000000,1.0,0.5
2006-11-10,1.0,-1.0,-0.500000,0.0,0.0
2006-11-13,1.0,0.0,-0.500000,0.0,-0.5
2006-11-14,0.0,0.5,-1.000000,0.0,-1.0
2006-11-15,-1.0,0.5,-1.000000,-1.0,-1.0
2006-11-16,0.0,0.0,-0.500000,-1.0,-1.0
2006-11-17,0.0,-0.5,0.000000,0.0,-1.0


In [101]:
def storePortfolioAllocation(portfolioKey, predictionDay, algorithmWeights, tickerAllocation, shouldReturn = False):
    toUpload = {}
    toUpload["portfolio"] = portfolioKey
    toUpload["predictionDay"] = predictionDay
    
    for item in algorithmWeights:
        toUpload["algo_" + item] = algorithmWeights[item]
    
    for item in tickerAllocation:
        toUpload["ticker_" + item] = tickerAllocation[item]
    
    ##UPLOAD ORGANISM OBJECT
    while True:
        try:
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            predictionHash = hashlib.sha224((str(portfolioKey) + " " + str(toUpload["predictionDay"])).encode('utf-8')).hexdigest()
            key = datastoreClient.key(params.portfolioAllocation, predictionHash) #NEED TO HASH TO ENSURE NON-OVERLAPPING PREDICTIONS
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            if shouldReturn == False:
                datastoreClient.put(organismToStore)
            else:
                return organismToStore
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [102]:
import hrpPortfolioOpt as hrp
def produceHRPPredictions(aggregateReturns, windowSize, startIndex, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize + startIndex
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            cov = (aggregateReturns[i-windowSize:i]).cov()
        weights = hrp.getHRP(cov, corr)
    #     display(weights)
    #     display(aggregateReturns[i+windowSize:i+windowSize+1])
        todayReturn = aggregateReturns[i:i+1] * weights
    #     display(todayReturn)
        sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
        hrpReturns = pd.concat([hrpReturns, sumReturn])
        thisWeights = pd.DataFrame([[weights[item] for item in weights.index]], index=sumReturn.index, columns=weights.index.tolist())
        historicalWeights = pd.concat([historicalWeights, thisWeights])
        i += 1
    return hrpReturns, historicalWeights

In [106]:
import importlib
importlib.reload(portfolio)

<module 'portfolio' from '/home/prohb125/walkforwardTrader/portfolio.py'>

In [107]:
def storeHistoricalAllocations(portfolioKey, modelsInPortfolio, historicalWeights, aggregatePredictions):

    aggregatePredictions = aggregatePredictions.dropna()
    allocationsToStore = []
    ##ITERATE THROUGH DAYS TO CALCULATE NET POSITION
    for i in range(len(historicalWeights)):
        netPosition = {}
        weights = historicalWeights.iloc[i]
        for model in modelsInPortfolio:
            if model.inputSeries.targetTicker not in netPosition:
                netPosition[model.inputSeries.targetTicker] = 0.0
            netPosition[model.inputSeries.targetTicker] += weights[str(model.describe())] * aggregatePredictions.loc[historicalWeights.index[i]][str(model.describe())]
        allocationsToStore.append(storePortfolioAllocation(portfolioKey, historicalWeights.index[i], weights.to_dict(), netPosition, shouldReturn=True))
    portfolio.storeManyItems(allocationsToStore)

In [109]:
##GENERATE WEIGHTS FOR PORTFOLIO
storeHistorical = False
allocationsToStore = []
for portfolioKey in modelsInPortfolio:
    hrpReturns, historicalWeights = produceHRPPredictions(aggregateReturns[[str(model.describe()) for model in modelsInPortfolio[portfolioKey]]], 22, 1500, True)
    if storeHistorical == True:
        storeHistoricalAllocations(portfolioKey, modelsInPortfolio[portfolioKey], historicalWeights, aggregatePredictions)
    print(portfolioKey, historicalWeights.iloc[-1])
    todayWeight = historicalWeights.iloc[-1]
    netPosition = {}
    for model in modelsInPortfolio[portfolioKey]:
        if model.inputSeries.targetTicker not in netPosition:
            netPosition[model.inputSeries.targetTicker] = 0.0
        netPosition[model.inputSeries.targetTicker] += weights[str(model.describe())] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
    print(portfolioKey, netPosition)
    allocationsToStore.append(storePortfolioAllocation(portfolioKey, portfolio.getToday(), weights.to_dict(), netPosition, shouldReturn=True))
portfolio.storeManyItems(allocationsToStore)

7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f ((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)    0.140269
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)    0.217790
((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)      0.125790
((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)      0.229397
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)     0.286753
Name: 2017-08-08 00:00:00, dtype: float64
7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f {'VO': 2010-11-29    0.387013
2010-11-30    0.339679
2010-12-01    0.383320
2010-12-02    0.370956
2010-12-03    0.375436
2010-12-06    0.374440
2010-12-07    0.398351
2010-12-08    0.395126
2010-12-09    0.395724
2010-12-10    0.390735
2010-12-13    0.386579
2010-12-14    0.386668
2010-12-15    0.376147
2010-12-16    0.384843
2010-12-17    0.412990
2010-12-20    0.411621
2010-12-21    0.356352
2010-12-22    0.351666
2010-12-23    0.359374
2010-12-27    0.363720
2010-12-28    0.368484
2010-12-29    0.367234
2010-12-30    0.369191
201

KeyboardInterrupt: 

In [3]:
def getPortfolioAllocations(portfolioKey):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.portfolioAllocation)
            query.add_filter('portfolio', '=', portfolioKey)
            retrievedPredictions = list(query.fetch())
            return retrievedPredictions
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))
    

In [5]:
getPortfolioAllocations("8d2f03a0255fb23c126417c182491b9c7d5d8181e30e4f51b67022ae")

[<Entity('walkforward_portfolio_allocation', '91e4ba0aff9a22a3ebe133b7e6bdeef6e078e3463cc30d956d364073') {'lastDataDayUsed': datetime.datetime(2017, 8, 10, 0, 0, tzinfo=<UTC>), 'ticker_XLY': -0.2943465183423877, "algo_((10, ('EWT', 33, None, None), 3, 'XLY'), 10, 50)": 0.7075988875679076, "algo_((10, ('EWT', 33, None, None), 5, 'XLY'), 10, 250)": 0.29240111243209244, 'portfolio': '8d2f03a0255fb23c126417c182491b9c7d5d8181e30e4f51b67022ae'}>]